In [37]:
import numpy as np
import pandas as pd
import sklearn
print(sklearn.__version__)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import chi2, RFE
from sklearn.metrics import mutual_info_score
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler


1.4.1.post1


In [3]:
#Cargar dataset
df = pd.read_csv('DatasetSinRellenar.csv', encoding="utf-8")

In [4]:
# Separar las columnas de características y etiquetas
X = df[['Clean_Resume', 'Common Skills']]
y = df['Category']

# Obtener el tipo de datos de una columna específica
print(df['Clean_Resume'].dtype)

# Convertir las columnas de características a texto
X['Clean_Resume'] = X['Clean_Resume'].astype(str)
X['Common Skills'] = X['Common Skills'].astype(str)

# Obtener el tipo de datos de una columna específica
print(df['Clean_Resume'].dtype)

# Aplicar TfidfVectorizer a las columnas de características
vectorizer = TfidfVectorizer()
X_tfidf = vectorizer.fit_transform(X['Clean_Resume'] + ' ' + X['Common Skills'])



object
object


C:\Users\pepeh\AppData\Local\Temp\ipykernel_10456\4258846628.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Clean_Resume'] = X['Clean_Resume'].astype(str)
C:\Users\pepeh\AppData\Local\Temp\ipykernel_10456\4258846628.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Common Skills'] = X['Common Skills'].astype(str)


In [5]:
# Divide el dataset en train y test
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.25)

In [32]:
# Selecciona características con chi-squared test
chi2_scores = chi2(X_train, y_train)
top_features_chi2 = np.argsort(chi2_scores[0])[-15:] #[1] obtener los valores de chi-cuadrado, [0] contiene los valores p(p =  indica la hipotesis 0.05 <)

In [11]:
# Convertir datos dispersos a matriz densa
X_train_dense = X_train.toarray()

mi_scores = []

# Calcular la MI para cada característica
for feature in X_train_dense.T:
    mi_scores.append(mutual_info_score(feature, y_train))

mi_scores = np.array(mi_scores)
top_features_mi = np.argsort(mi_scores)[-10:]

c:\Users\pepeh\Documents\GitHub\TT2\.venv\Lib\site-packages\sklearn\metrics\cluster\_supervised.py:66: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and multiclass values for target
  warnings.warn(msg, UserWarning)
c:\Users\pepeh\Documents\GitHub\TT2\.venv\Lib\site-packages\sklearn\metrics\cluster\_supervised.py:66: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and multiclass values for target
  warnings.warn(msg, UserWarning)
c:\Users\pepeh\Documents\GitHub\TT2\.venv\Lib\site-packages\sklearn\metrics\cluster\_supervised.py:66: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and multiclass values for target
  warnings.warn(msg, UserWarning)
c:\Users\pepeh\Documents\GitHub\TT2\.venv\Lib\site-packages\sklearn\metrics\cluster\_supervised.py:66: UserWarning: Clustering metrics expects discrete values but received continuous values for la

In [27]:
# Selecciona características con RFE
selector = RFE(estimator=SVC(kernel="linear"), n_features_to_select=10)
selector.fit(X_train, y_train)
top_features_rfe = selector.support_
print(selector.ranking_)


[10723  6866 12385 ... 11344 10123 12636]


In [33]:
# Imprime las características seleccionadas
print(f"Top features chi2: {top_features_chi2}")
print(f"Top features mutual information: {top_features_mi}")
print(f"Top features RFE: {top_features_rfe}")

Top features chi2: [ 5530  5528 13519 12632  5526 12633  3952  5518  9484  2340  1402 11093
  5514  3953  3933]
Top features mutual information: [ 9196 12156  7789 12367  3305 12042 12851  3645  3578 11672]
Top features RFE: [False False False ... False False False]


In [34]:
# Aplicar esta selección a tus conjuntos de datos
X_train_selected = X_train[:, top_features_chi2]
X_test_selected = X_test[:, top_features_chi2]

In [38]:
# Paso 1: Estandarizar los datos
# Es importante estandarizar los datos antes de aplicar PCA para asegurar que cada característica contribuya equitativamente
scaler = StandardScaler()
X_standardized = scaler.fit_transform(X_train_selected)

ValueError: Cannot center sparse matrices: pass `with_mean=False` instead. See docstring for motivation and alternatives.

In [ ]:
# Paso 2: Aplicar PCA
# Puedes ajustar el número de componentes (n_components) según tus necesidades o dejarlo sin definir para conservar todos
pca = PCA()
pca.fit(X_standardized)

In [ ]:
# Calcular la varianza explicada acumulativa
varianza_explicada_cumulativa = np.cumsum(pca.explained_variance_ratio_)

In [ ]:
# Graficar la varianza explicada acumulativa
plt.figure(figsize=(10, 5))
plt.plot(range(1, len(varianza_explicada_cumulativa)+1), varianza_explicada_cumulativa, marker='o', linestyle='-')
plt.title('Análisis del Codo para PCA')
plt.xlabel('Número de Componentes Principales')
plt.ylabel('Varianza Explicada Acumulativa')
plt.grid(True)
plt.show()